# <center>**Import necessary libraries**</center>


In [977]:
import pathlib
import pandas as pd
import math
import design_functions.rebar_information as rebar_func

## <center>Import and read the desired beam flexural and shear reinforcement excel file from ETABS</center>

In [978]:
excel_file = pathlib.Path('excel_files\example_analysis.xlsx')
initial_df = pd.read_excel(excel_file, sheet_name=None)
pd.set_option('display.max_rows', None)

## <center>Because the excel sheet has two sheets, flexural and shear, they need to be read individually</center>

## <center>Delete unnecessary rows and columns for simplicity</center>

V1 of respective df's = first two rows are deleted<br>
V2 of respective df's = unique name, story, negative moment, negative combo, positive moment, and positive combo columns are removed.<br>
V3 of respective df's = each unique beam is consolidated to extract information such as section size, unqiue name.<br>
V4 of respective df's = ETABs section column is removed.

In [979]:
v1_flexural_df = initial_df['Sheet1'].drop([0,1])
v1_shear_df = initial_df['Sheet2'].drop([0,1])

In [980]:
v2_flexural_df = v1_flexural_df.drop(['Unnamed: 2', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 8', 'Unnamed: 9'], axis=1)
#insert v2_shear_df once understood which columns to remove.

In [981]:
v3_flexural_df = v2_flexural_df.iloc[::3]
v3_shear_df = v1_shear_df.iloc[::3] #remember to change variable to v2 once it has been created

## <center>Create width and depth columns to utilise in calculations</center>

In [982]:
v3_flexural_df.insert(3, 'Width (mm)', None)
v3_flexural_df.insert(4, 'Depth (mm)', None)

In [983]:
#this function cleans the cell of unnamed: 3 column to provide the width of each respective beam.
def clean_width_dimensions(width):
    width_list = list(width) # turn string into list of individual indexes
    width_list = [el.lower() for el in width_list] #use list comprehension to turn list into lower case values
    excluded_values = ['p', 't', 'b', '-', '_', 'c', '/'] #create list of excluded indices
    v1_width_list = [ex for ex in width_list if ex not in excluded_values] #use list comprehension to return list excluding indices
    index_list = v1_width_list.index('x') #index the list to x to retrieve required width
    v2_width_list = v1_width_list[:index_list] #slice the width list to the index x
    true_width = ''.join(v2_width_list) #join the list into a string
    return int(true_width) #turn string into int so it can be used in other functions

In [984]:
v3_flexural_df.loc[:, 'Width (mm)'] = v3_flexural_df['Unnamed: 3'].apply(clean_width_dimensions)

In [985]:
#this function cleans the cell of unnamed: 3 column to provide the depth of each respective beam.
#this function follows the same steps as clean_width_dimensions function
def clean_depth_dimensions(depth):
    depth_list = list(depth)
    depth_list = [el.lower() for el in depth_list]
    excluded_values = ['p','t', 'b', '-', '_', 'c', '/']
    v1_depth_list = [ex for ex in depth_list if ex not in excluded_values]
    index_list = v1_depth_list.index('x')
    v2_depth_list = v1_depth_list[1+index_list:-4]
    true_depth = ''.join(v2_depth_list)
    return int(true_depth)

In [986]:
v3_flexural_df.loc[:, 'Depth (mm)'] = v3_flexural_df['Unnamed: 3'].apply(clean_depth_dimensions)

## <center>With the width and depth defined, remove the ETABS section, location, and rebar area columns</center>

In [987]:
v4_flexural_df = v3_flexural_df.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 7', 'Unnamed: 10'], axis=1)
#insert v4_shear_df once understood which columns to remove.

In [988]:
v4_flexural_df = v4_flexural_df.rename(columns={'Unnamed: 1': 'ETABS beam ID'})

## <center>Create Dataframe to store beam reinforcement schedule</center>

In [989]:
# Create dictionary which reflects column headings
columns = pd.MultiIndex.from_tuples([
    ('Storey', ''),
    ('Etabs ID', ''),
    ('Dimensions', 'Width (mm)'),
    ('Dimensions', 'Depth (mm)'),
    ('Bottom Reinforcement', 'Left (BL)'),
    ('Bottom Reinforcement', 'Middle (B)'),
    ('Bottom Reinforcement', 'Right (BR)'),
    ('Top Reinforcement', 'Left (TL)'),
    ('Top Reinforcement', 'Middle (T)'),
    ('Top Reinforcement', 'Right (TR)'),
    ('Shear links', 'Left (H)'),
    ('Shear links', 'Middle (J)'),
    ('Shear links', 'Right (K)'),
    ('Side Face Reinforcement', '')
])
beam_schedule_df = pd.DataFrame(columns=columns)

## <center>For each index of the v4 respective df, populate the main beam_schedule_df with the necessary information</center>

In [990]:
beam_schedule_df['Storey'] = v4_flexural_df['TABLE:  Concrete Beam Flexure Envelope - ACI 318-19']
beam_schedule_df['Etabs ID'] = v4_flexural_df['ETABS beam ID']
beam_schedule_df['Dimensions', 'Width (mm)'] = v4_flexural_df['Width (mm)']
beam_schedule_df['Dimensions', 'Depth (mm)'] = v4_flexural_df['Depth (mm)']
beam_schedule_df.head()

Storey Etabs ID Dimensions            Bottom Reinforcement             \
                   Width (mm) Depth (mm)            Left (BL) Middle (B)   
2      P4     B213        300        500                  NaN        NaN   
5      P4     B215        300        500                  NaN        NaN   
8      P4     B232        600        600                  NaN        NaN   
11     P4     B235        300        500                  NaN        NaN   
14     P4     B237        300        500                  NaN        NaN   

              Top Reinforcement                       Shear links             \
   Right (BR)         Left (TL) Middle (T) Right (TR)    Left (H) Middle (J)   
2         NaN               NaN        NaN        NaN         NaN        NaN   
5         NaN               NaN        NaN        NaN         NaN        NaN   
8         NaN               NaN        NaN        NaN         NaN        NaN   
11        NaN               NaN        NaN        NaN         NaN        NaN   
14        NaN               NaN        NaN        NaN         NaN        NaN   

             Side Face Reinforcement  
   Right (K)                          
2        NaN                     NaN  
5        NaN                     NaN  
8        NaN                     NaN  
11       NaN                     NaN  
14       NaN                     NaN

## <center>Create a dictionary of each unique beam from v3 df, and associate the relevant top / bottom rebar requirement with each </center>

In [991]:
bottom_beam_rebar = {}
values_per_key = 3
values_counter = 0
for i, row in v3_flexural_df.iterrows():
    key = row['Unnamed: 1']
    values = v2_flexural_df.iloc[values_counter:values_counter+values_per_key]['Unnamed: 7'].tolist()
    bottom_beam_rebar[key] = values
    values_counter += values_per_key

In [992]:
top_beam_rebar = {}
values_per_key = 3
values_counter = 0
for i, row in v3_flexural_df.iterrows():
    key = row['Unnamed: 1']
    values = v2_flexural_df.iloc[values_counter:values_counter+values_per_key]['Unnamed: 10'].tolist()
    top_beam_rebar[key] = values
    values_counter += values_per_key